In [76]:
import argparse
import torch
import os
from torchvision import transforms, utils, datasets, models
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from collections import OrderedDict
import time
from PIL import Image
import pandas as pd
import numpy as np
import glob
import copy
import shutil
import sys
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

%matplotlib inline
np.random.seed(42)
torch.manual_seed(42)
from keras.models import Model

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached tornado-4.5.3-cp39-cp39-win_amd64.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached idna-2.8-p

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1130 lines of output]
  C:\Users\vip\AppData\Local\Temp\pip-install-d0rwmav8\pandas_413cb572f21642c0883e11f59c6f3f19\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-39\pandas
  creating build\lib.win-amd64-cpython-39\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-39\pandas\api
  creating build\lib.win

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached tornado-4.5.3-cp39-cp39-win_amd64.whl
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using c

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1130 lines of output]
  C:\Users\vip\AppData\Local\Temp\pip-install-0b1vjda5\pandas_bbb5fe78890a411eb6ce9c3fe7dcf45f\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-39\pandas
  creating build\lib.win-amd64-cpython-39\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-39\pandas\api
  creating build\lib.win

# Loading dataset

In [59]:
dir = r'C:\Users\vip\Downloads\Dataset'

def remove_stray_files(image_path):
  """
  Remove the House-dataset directory everytime the script is run
  to avoid the chance of stray images being present in the directory
  :param image_path - a directory specifying the source of images
  return - None
  """
  try:
    shutil.rmtree(image_path)
  except OSError:
    # directory  doesn't exist
    pass

remove_stray_files(dir)

# train test split

In [60]:
def split_into_train_test_valid(image_path, split_ratio=0.1):
  """
  Split the input images into train, test and validation
  house ids
  :param image_path - a directory specifying the source of images
  :param split_ratio - a float specifying the proportion of test images
  return - a list of house ids for train and test
  """
  house_ids = set()
  for file in os.listdir(image_path):
    if file.endswith('.jpg'):
      plant_ids.add(int(file.split('_')[0]))
  plant_ids = list(house_ids)    
  np.random.shuffle(house_ids)    
  num_plant = len(house_ids)
  split_num = int(num_houses * split_ratio)
  test_ids = house_ids[:split_num]
  train_ids = house_ids[split_num:]
  num_plant = len(train_ids)
  split_num = int(num_houses * split_ratio)
  valid_ids = train_ids[:split_num]
  train_ids = train_ids[split_num:]

  return train_ids, test_ids, valid_ids

In [61]:
def make_dirs_copy_images(image_path, *plant_ids):
  """
  Make train, test and validation directories
  and copy the files from `image_path` to their
  respective directories using `house_ids`
  :param image_path - a directory specifying the source of images
  :param house_ids - a list of house ids for train, test and validation images
  return - None
  """
  try:
    os.makedirs(os.path.join(image_path, 'train'), exist_ok=True)
  except FileExistsError:
    pass
  
  try:
    os.makedirs(os.path.join(image_path, 'test'), exist_ok=True)
  except FileExistsError:
    pass
  
  try:
    os.makedirs(os.path.join(image_path, 'valid'), exist_ok=True)
  except FileExistsError:
    pass
  
  train_ids, test_ids, valid_ids = house_ids
  for file in os.listdir(image_path):
    house_id = file.split('_')[0]
    if file.endswith('.jpg'):
      if int(house_id) in train_ids:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path, 'train', file))
      elif int(plant_id) in test_ids:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path, 'test', file))
      else:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path, 'valid', file))

# Class implementation

In [62]:
def load_data(data_dir, data, df, plant_ids, size=128, image_transform=None):
    """
    Load the train, test and validation datasets from their respective
    directories
    :param data_dir - a string specifying the base directory
    :param data - a string specifying the type of dataset
    :param df - a DataFrame containing the housing information
    :param house_ids - a list of house ids
    :param size - an integer specifying the size of the image
    :param image_transforms - a transform object for that data type
    return - a dictionary mapping the data type to the location of the image dataset
    """
    return {data: PlantDataset(df, plant_ids, data_dir, data, size, transform=image_transform)}

def transform_images(size=224):
  """
  Apply transformations separately to the train, test and validation
  datasets
  :param size - an integer specifying the size of the image
  return - a dictionary mapping the type of dataset to its transformaions
  """
  data_transforms = {
                    'train': transforms.Compose([
                                                  transforms.ColorJitter(
                                                                        brightness=0.2*torch.rand(1).item(), 
                                                                        hue=0.1*torch.rand(1).item(), 
                                                                        saturation=0.3*torch.rand(1).item(),
                                                                        contrast=0.3*torch.rand(1).item()
                                                                        ),
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                      )
                                                ]),
                      'valid': transforms.Compose([
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                      )
                                                 ]),
                      'test': transforms.Compose([
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                    )
                                                 ])
  }
  return data_transforms
  
def data_loader(image_datasets, data, size, shuffle=False):
    """
    Create generator objects from the image datasets
    :param image_datasets - an ImageFolder dataset object for that data type
    :param data - a string specifying the dataset type
    :param size - an integer specifying the size of the batch
    :param shuffle - a bool
    return a dictionary mapping the data type to the dataloader object
    """
    
    return {data: torch.utils.data.DataLoader(image_datasets, batch_size=size, shuffle=shuffle)}

In [63]:
def build_model(arch, num_classes, hidden_units=1024):
  """
  Load a pretrained model with only the final layer
  replaced by a user-defined classifier
  :param arch - a string specifying the type of model architecture
  :param num_classes - an integer specifying the number of class labels
  :param hidden_units - an integer specifying the size
  return - a pretrained model with a user-defined classifier
  """
  in_features = 0
  try:
        # model = eval("models." + arch + "(pretrained=True)")
        model = models.__dict__[arch](pretrained=True)
  except:
      raise Exception('Invalid architecture specified')
    
  # Freeze parameters as only the final layer is being trained
  for param in model.parameters():
      param.require_grad = False
  # extract the last layer in the model
  last_layer = list(model.children())[-1]
  if isinstance(last_layer, nn.Sequential):
      count = 0
      for layer in last_layer:
          if isinstance(layer, nn.Linear):
              # fetch the first of the many Linear layers
              count += 1
              in_features = layer.in_features
          if count == 1:
              break
  elif isinstance(last_layer, nn.Linear):
      in_features = last_layer.in_features
  # define the new classifier
  classifier = nn.Sequential(OrderedDict([
                          ('bc1', nn.BatchNorm1d(in_features)),
                          ('relu1', nn.ReLU()),
                          ('fc1', nn.Linear(in_features, num_classes, bias=True)),
  ]))
  # replace the existing classifier in thelast layer with the new one
  if model.__dict__['_modules'].get('fc', None):
      model.fc = classifier
  else:
      model.classifier = classifier

  return model

In [72]:
def train_model(model, trainloader, validloader, criterion, optimizer, scheduler, epochs, diff_lr=False, device='cpu'):
  
  """
  Train the model and run inference on the validation dataset. Capture the loss
  of the trained model and validation model. Also display the accuracy of the
  validation model
  :param model - a pretrain model object
  :param trainloader - a generator object representing the train dataset
  :param validloader - a generator object representing the validation dataset
  :param criterion - a loss object
  :param optimizer - an optimizer object
  :param scheduler - a scheduler object that varies the learning rate every n epochs
  :param epochs - an integer specifying the number of epochs to train the model
  :param diff_lr - a boolean specifying whether to use differential learning rate
  :param device - a string specifying whether to use cuda or cpu
  return a trained model with the best weights
  """
  start = time.time()
  print_every = 50
  steps = 0
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  valid_loss_min = np.Inf
  training_loss, validation_loss = [], []
  for epoch in range(epochs):
      lr_used = 0
      if diff_lr:
        for param in optimizer.param_groups:
          if param['lr'] > lr_used:
            lr_used = param['lr']
        print('learning rate being used {}'.format(lr_used))      
      running_loss = 0
      # train_acc = 0
      scheduler.step()
      model.train()
      for idx, (images, prices) in enumerate(trainloader):
          steps += 1
          images, prices = images.to(device), prices.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward pass and backward pass
          output = model(images)
          loss = criterion(prices, output)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * images.size(0)
          # ps = torch.exp(output)
          # train_acc += (ps.max(dim=1)[1] == labels.data).type(torch.FloatTensor).mean()

#            if steps % print_every == 0:
      # Turn off gradients for validation, saves memory and computations
      with torch.no_grad():
          valid_loss = validation(model, validloader, criterion, device)
#           scheduler.step(test_loss)

      # if test_accuracy > best_acc:
      if valid_loss < valid_loss_min:
          valid_loss_min = valid_loss
          best_model_wts = copy.deepcopy(model.state_dict())

      print("Epoch: {}/{}... ".format(epoch+1, epochs),
        "Train RMSLE loss: {:.4f}".format(running_loss/len(trainloader.dataset)),
        "Validation RMSLE loss: {:.4f}".format(valid_loss/len(validloader.dataset)),
        )
      # save the losses
      training_loss.append(running_loss/len(trainloader.dataset))
      validation_loss.append(valid_loss/len(validloader.dataset))
      running_loss = 0       
      
  print('Best validation RMSLE loss is {:.4f}'.format(valid_loss_min/len(validloader.dataset)))
  print('Time to complete training {} minutes'.format((time.time() - start) / 60))
  model.load_state_dict(best_model_wts)
  return model, training_loss, validation_loss

In [73]:
def validation(model, validloader, criterion=None, device='cpu'):
    """
    Compute loss on the validation dataset
    :param model - a pretrained model object
    :param validloader - a generator object representing the validataion dataset
    :param criterion - a loss object
    :param device - a string specifying whether to use cuda or cpu
    return a tuple of loss and accuracy
    """
    valid_loss = 0
    model.eval()
    for images, prices in validloader:

        images, prices = images.to(device), prices.to(device)
        output = model(images)
        valid_loss += criterion(prices, output).item() * images.size(0)
    
    return valid_loss

In [82]:
def main():
  image_dir = r'C:\Users\vip\Downloads\Dataset'
  
  train_ids, test_ids, valid_ids = split_into_train_test_valid(image_dir)
  make_dirs_copy_images(image_dir, train_ids, test_ids, valid_ids)

  #houses_paths = load_images(df, image_dir, 'train')

  train_batch_size= 32
  test_batch_size = 1
  valid_batch_size = 32
  img_size = 128
  transform_img_size = 224

  image_datasets = {}
  dataloader = {}
  image_transforms = transform_images(size=transform_img_size)
  try:
    for data_type,plant_ids, batch_size, shuffle in zip(['train', 'test', 'valid'],
                         [train_ids, test_ids, valid_ids],
                         [train_batch_size, test_batch_size, valid_batch_size],
                         [True, False, True]):
      image_datasets.update(load_data(image_dir, data_type, df, house_ids, img_size, image_transforms[data_type]))
      dataloader.update(data_loader(image_datasets[data_type], data_type, batch_size, shuffle))
  except:  
      raise Exception('Loading of dataset failed...')

  arch = 'resnet50'
  hidden_units = 64
  lr = 0.0001
  epochs = 50
  gpu = True
  num_classes = 1 # for regression

  model = build_model(arch, num_classes, hidden_units)
  print(model)
  criterion = RMSLELoss()
#  criterion = torch.nn.L1Loss()
#  criterion = torch.nn.MSELoss()
  optim_ = optimizer(model, lr)
  #exp_lr_scheduler = lr_scheduler.StepLR(optim_, step_size=5, gamma=0.1)
  exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optim_, T_max=epochs)
  # default
  device = "cpu"
  if gpu:
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print('Use device:', device)
  model = model.to(device) 
  print('Training begins...')
  model, training_loss, validation_loss = train_model(model, dataloader['train'], 
                                                      dataloader['valid'], criterion, 
                      optim_, exp_lr_scheduler, epochs, False, device) 
  
  test_loss = check_rmsle_on_test(model, dataloader['test'], criterion, device='cuda')
  print('Test set RMSLE loss is {:.4f}'.format(test_loss))
    
  predicted_prices = predict_prices(model, dataloader['test'], criterion,
                   device='cuda', scale_factor=1e6)
  plant_predicted_prices = pd.DataFrame({'house_ids': test_ids, 
                'predicted_prices': predicted_prices})
  
  plot_losses(training_loss, validation_loss, epochs)
  
  return model, plant_predicted_prices
from keras.models import Model